# Tree Based models

## Overview

Tree-based models are the models that use **Decision Trees** as their base algorithm. decision tree itself isn't that powerful model but other tree-based models like **Random Forest** and **Boosting Trees** are one of the most powerful algorithms out there. so in order to learn these algorithms first, we have to learn the decision tree. so first we will talk about decision trees and how to implement them from scratch, then we will talk about the ensemble methods.

## Decision Tree

in general, a Decision Tree (DT) makes a statement and then makes a decision based on whether or not the statement is True or False. usually, it is assumed that if the statement is True we go to the left and if it is False, we go to the right (this is the assumption we want to use from now on and also in our code). like other algorithms, if a DT classifies things, it is called **Classification Tree** and when a DT predicts numeric values, it is called **Regression Tree**.<br>
These two types of trees have so much in common but have some differences too, so we gonna talk about them separately. but let's first talk about some terminology that we want to use when talking about tree-based models.<br>
**Terminology:**<br>
   * **Root node** or **the Root**: the first statement that our tree decides to make about our data.
   * **Internal nodes** or **Branches**: other statements throughout our tree construction.
   * **Leaf Nodes** or **Leaves**: nodes without any statement. these are the nodes we use to predict.
   * **Depth**: number of times our tree splits.
   * **Pure** or **Impure** leaf: in the Classification Tree if a leaf only contains one category it will be called pure if not, it will be called impure.<br>
   
**Classification Trees**:<br>
the classification trees, are used when we want to classify things, so in our leaf nude we will have classes. as we said before, if in leaf nodes we just have one class category then we call the leaf pure otherwise it is impure. so now the question is, how do we construct our tree? how do we choose our statement? for choosing our statement we need some criteria to compare different statements and then choose one with respect to our criteria. in classification trees we will use **Gini Index** or **Cross-Entropy**, and choose the statement that has smallest Gini or Entropy. lower criteria mean we separate more classes using that statement. so we have to search over all possible statements and choose the one with the smallest criteria. for categorical variables, we check all the possible questions but for numerical variables, we have to do something else. so we sort our feature column values and use the mean value between unique values and use that threshold to make a statement and choose the one with smallest criteria. for each split, we have to check all the possible statements for all columns and choose the smallest criteria. after that, we split our data using that statement. for the next split now we have 2 separate datasets so again we check all possible statements and use one with the smallest criteria. but before we compare criteria we have to do one more thing and use some weight because these 2 datasets probably won't have the same number of samples so we assign some weight with respect to their sample size, so we will call that weighted criteria **Weighted Gini** or **Weighted Entropy**. we use the sum of these 2 weighted criteria to represent our statement criteria. we continue these steps until we reach pure leaves and after that, we won't split. so as you can see we can learn our data perfectly or in other words, we will overfit.<br>
in the **prediction phase**, we move from top to bottom with respect to our sample feature values and we end up in some leaf, so the class in that leaf will be our prediction. if the leaf isn't pure (later) we will use the majority class as our prediction.<br>
to **prevent overfitting** we can use these techniques:<br>
   * Pruning
   * Maximum depth
   * Maximum leaf nodes
   * minimum sample split
   * minimum impurity decrease<br>

**Notes**:
   * Gini index is between 0 and 0.5 and entropy is between 0 and 1 (from best separation to worst)
   * We have to treat these overfitting techniques as hyperparameters
   * by using these techniques some leaf wont be pure.


**Regression Trees**:<br>
Regression trees are like classification trees in various ways but they have some differences too. in leaves, we will have continuous values. for criteria we are using **Mean Square Error** or **Mean Absolute Error** (and again use minimum for choosing our statement). in the prediction phase we will use the mean of values in leaf as our prediction.<br>

**Some Pros and Cons of DT**:<br>
   * Trees are non-linear and non-parametric models, which means they can grow as complex as necessary for any given dataset.
   * They don't need preprocessing and can be trained with categorical data.
   * They are interpretable.
   * Without using any prohibition, our tree will overfit the training data.
   * Trees are unstable, which means if we have a slight change in our dataset we may end up with a completely different tree.
   * Generally they have low prediction accuracy.<br>

now let's talk about how we can implement DT from scratch.

### Implementing decision Tree from scratch

#### Psudo code for classification tree

here I will use a recursive function to build my tree (I don't know if there is another way to implement it). because it is a recursive function so it has a base case and our function will call itself. the base case is when we want our function to stop repeating itself and here it is when we reach our leaf so we need a condition that defines when we want to stop from splitting or in other words we reach our leaf.

1. check if the data is pure (this is where we define our base case. we can use other base cases too).
2. if the base case is True, then treat the data as a leaf and predict. (by using the vast majority for classification and averaging for regression).
3. if the base case is False, then treat these data as a node and split it with the next steps:<br>
   * Find all potential splits.
   * find the best split from all the potential splits using our criteria.
   * split the node with our best split.
4. now that we split our data, perform steps 1 to 3 again for each split data. (this is our recursive part).

#### Some Notes

* here we can't use categorical variables so we have to change them in advance.
* the algorithm will continue until all the split data reach our base case i.e becoming leaf. but we add two more base case to prevent overfitting.
* The data we are using is our breast cancer dataset but the feature columns and label column are in the same data and the label column is our last column.
* My goal for writing the codes is to write them as easy as possible to not distract by code complexity. so I try not to use classes or even functions but here at the end, I have to use them because it is a recursive algorithm :) so I decide to use function through all the code and it is make everything much simpler and we can go through our algorithm step by step and not lost in the middle.
* here I will code the classification tree but I also write the functions we need for regression.
* The code is inspired by https://www.youtube.com/c/SebastianMantey youtube channel but I try to make it simpler and cleaner and add some other think or not use some part of the code that explained in the channel.
* I'm a beginner in coding and it should be a better way to write the code, so try it your way :).

#### code

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
data,target= load_breast_cancer(return_X_y=True, as_frame=True) 

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data,target, test_size=0.2, random_state=42,stratify=target)

In [4]:
data_train = pd.concat([X_train,y_train],axis=1)
data_test = pd.concat([X_test,y_test],axis=1)
data_train = np.array(data_train)
data_test = np.array(data_test)

##### Helper Functions

1. checking the purity of data

In [5]:
def check_purity(data):
    labels = data[:,-1]
    unique_class = np.unique(labels)
    if len(unique_class) == 1:
        return True
    else:
        return False

2.1 leaf output for classification

In [6]:
def leaf_classification(data):
    labels = data[:,-1]
    unique_classes, count_unique_classes = np.unique(labels,return_counts=True)
    index = np.argmax(count_unique_classes)
    return unique_classes[index]

2.2 leaf output for regression

In [7]:
def leaf_regression(data):
    output = data[:,-1]
    return np.mean(output)

3. Potential split for numerical columns

In [8]:
def get_potential_splits(data):
    potential_splits = {}
    n = data.shape[1] # number of columns for features
    for column_index in range(n-1): ## if data and labels are in the same df then n-1
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)
        for i in range(1, len(unique_values)):
            average_unique_values = (unique_values[i] + unique_values[i-1])/2
            potential_splits[column_index].append(average_unique_values)
    return potential_splits


4. Split Function

In [9]:
def split_data(data,split_column,split_value):
    split_column_value = data[:, split_column]
    data_below = data[split_column_value <= split_value]
    data_above = data[split_column_value > split_value]
    return data_below, data_above

5.1 Overall entropy of leaves for classification

In [10]:
def entropy(data):
    labels = data[:,-1]
    counts = np.unique(labels,return_counts=True)[1]
    probabilites = counts/np.sum(counts)
    entropy = np.sum(-probabilites*np.log2(probabilites))
    return entropy

In [11]:
def calculate_overall_entropy(data_below,data_above):
    data_points = len(data_below) + len(data_above)
    w_data_below = len(data_below)/data_points
    w_data_above = len(data_above)/data_points
    overall_entropy = (w_data_below * entropy(data_below)) + (w_data_above * entropy(data_above))
    return overall_entropy

5.2 overall gini of leaves for classification

In [12]:
def gini(data):
    labels = data[:,-1]
    counts = np.unique(labels,return_counts=True)[1]
    probabilites = counts/np.sum(counts)
    gini = 1 - np.sum(probabilites**2)
    return gini

In [13]:
def calculate_overall_gini(data_below,data_above):
    data_points = len(data_below) + len(data_above)
    w_data_below = len(data_below)/data_points
    w_data_above = len(data_above)/data_points
    overall_gini = (w_data_below * gini(data_below)) + (w_data_above * gini(data_above))
    return overall_gini

5.3 Overall mean square error of leaves for regression

In [14]:
def mse(data):
    actual_values = data[:,-1]
    mean_actual_values = np.mean(actual_values)
    mse = np.mean((actual_values-mean_actual_values)**2)
    return mse

In [15]:
def calculate_overall_mse(data_below,data_above):
    data_points = len(data_below) + len(data_above)
    w_data_below = len(data_below)/data_points
    w_data_above = len(data_above)/data_points
    overall_mse = (w_data_below * mse(data_below)) + (w_data_above * mse(data_above))
    return overall_mse

6.1 Determine best split value and column for classification

In [16]:
def best_split_classification(data, potential_splits): # or we can use calculate_overall_gini
    
    overall_entropy = 1000 # we know that entropy is between 0 and 1 so we choose a high number so our code at least executes once then overwrite it with a new value
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value = value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above) 
            if current_overall_entropy < overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
            
    return best_split_column, best_split_value

6.2 Determine best split value and column for regression

In [17]:
def best_split_regression(data, potential_splits):
    
    first_iteration = True
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value = value)
            current_overall_mse = calculate_overall_mse(data_below, data_above) 
            if first_iteration or current_overall_mse < best_overall_mse:
                best_overall_mse = current_overall_mse
                best_split_column = column_index
                best_split_value = value
            
    return best_split_column, best_split_value

##### DT Algorithm

In [18]:
def DT_algorithm(data,counter=0,min_sample=2,max_depth=5):
    if (check_purity(data)) or (len(data) < min_sample) or (counter==max_depth):
        classification = leaf_classification(data)
        return classification
    else:
        counter+=1
        potential_splits = get_potential_splits(data)
        split_column, split_value = best_split_classification(data, potential_splits)
        data_below, data_above = split_data(data,split_column, split_value)
        
        question = '{} <= {}'.format(split_column,split_value)
        subtree = {question : []}
        
        yes_answer = DT_algorithm(data_below,counter,min_sample,max_depth)
        no_answer = DT_algorithm(data_above,counter,min_sample,max_depth)
        
        subtree[question].append(yes_answer)
        subtree[question].append(no_answer)
        return subtree
        

In [19]:
%%time
tree = DT_algorithm(data_train,max_depth=3,min_sample=20)

CPU times: total: 3.62 s
Wall time: 3.63 s


In [20]:
tree

{'20 <= 16.795': [{'27 <= 0.13595': [{'13 <= 38.605000000000004': [1.0, 1.0]},
    {'1 <= 20.299999999999997': [1.0, 0.0]}]},
  {'11 <= 0.47315': [1.0, {'26 <= 0.1907': [1.0, 0.0]}]}]}

##### Prediction with our tree

In [21]:
def prediction(data,tree):
    question = list(tree.keys())[0]
    feature_name,comparison_operator,value = question.split() # it will split the string into several split with string type
    
    if data[int(feature_name)] <= float(value): # we have to change the splits into numbers
        answer = tree[question][0]
    else:
        answer = tree[question][1]
    if not isinstance(answer, dict): # will return true of answer is dict, if is not dict it is our leaf i.e our answer if is dict, its another node with split
        return answer
    else:
        return prediction(data,answer)

In [22]:
predictions = []
for i in range(len(data_test)):
    predictions.append(prediction(data_test[i],tree))

In [23]:
accuracy = np.sum(predictions==data_test[:,-1])/len(data_test[:,-1])
accuracy

0.9473684210526315

### Implementing Decision Tree with Scikit Learn

In [24]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [25]:
data,target= load_breast_cancer(return_X_y=True) 
X_train, X_test, y_train, y_test = train_test_split(data,target, test_size=0.2, random_state=42,stratify=target)

In [26]:
tree = DecisionTreeClassifier(criterion='entropy',min_samples_split=20)
tree.fit(X_train,y_train)
tree.score(X_test,y_test)

0.9473684210526315

well, we get the same accuracy, it feels good, right? :)

## Ensemble Technique

The idea here is instead of asking a question from one person (even an expert one or in our topic, the algorithm with high accuracy), we can ask our question to thousands of random people (random and independent from each other which means they won't affect each other answers) and the majority vote or someother combination their answers, is our answer. we will often get better predictions to compare with the best individual predictors. the group of predictors is called **Ensemble** so this technique is called **Ensemble Learning**. therefore we can train our dataset with different algorithms (e.g LR, SVM, DT) and use voting or averaging between the result of these algorithms and use it as our final prediction, we usually get better answers. but besides that, there are three methods of ensembling:<br>
   * Bagging (Bootstrap AGGregation)
   * Boosting
   * Stacking (Stacked Generalization)

in **Bagging** we use **Bootstrap** sampling which means random sampling subsets of our original dataset with replacement same size as our dataset and train our model on them and use voting or averaging for prediction. with this model, we can have the same bias with lower variance compared to the individual model so it is a good fit for a high variance algorithm (like a decision tree).<br>
in **Boosting** we combine several **Weak Learners** into a strong learner. the weak learner means models that do slightly better than random chance i.e having accuracy slightly bigger than 50%. the general idea of most boosting methods is to train predictors sequentially, and each one trying to correct its predecessor.<br>
in **Stacking** like simple ensembling, we use different algorithms but instead of using voting we assign some weight to each model and determine which algorithm we should trust more i.e have more impact on our final prediction. so these weights should learned through the algorithm.<br>
in the following, we want to talk about the combination of these ensemble methods with DT and make two of the most powerful models.

## Random forest

Random forest is the combination of bagging and decision tree. so the idea here is to create new datasets using bootstrap sampling and train decision tree on each of the bootstraped dataset. so:<br>
1. Create a bootstrapped dataset, which means sampling randomly with replacement.
2. create a decision tree using the bootstrapped dataset, but instead of using all the feature columns choose only random subsets of feature columns and build your tree with them.
3. repeat steps 1 and 2 bunch of time.<br>

by using bootstrap sampling and subset of our columns to build a tree, we will have wide varity of trees (that reasonably independent from each other). now in prediction phase, we run our new sample through all the trees and use vast majority of answers as our prediction for classification and average for regression.<br>
**NOTES:**<br>
   * in random forest beside all the hyperparameters in DT, now we have number of trees or number of estimators (maybe something between 100-1000).
   * number of random columns is also can be treated as hyperparameter or we can use sqrt(#columns).
   * when we are bootstrapping our dataset typically 1/3 of our original data doesn't endup in the bootsrapped dataset. these part of dataset called **Out Of Bag** dataset. we can use these out of bag datasets as cross validation set and tune our random forrest with it.

### Implementing Random forest in Scikit Learn

In [27]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [28]:
data,target= load_breast_cancer(return_X_y=True) 
X_train, X_test, y_train, y_test = train_test_split(data,target, test_size=0.2, random_state=42,stratify=target)

In [29]:
RF = RandomForestClassifier(n_estimators=200,criterion='gini')
RF.fit(X_train,y_train)
RF.score(X_test,y_test)

0.956140350877193

## Boosting Trees

in general, as we said before, in boosting we have weak learners (here, shallow tree) and will use a bunch of them to create a strong model. because we have weak learners, like Rf, averaging them won't do much for us so instead in each step, somehow we have to specify the mistake we make and use the next learner and try to overcome those mistakes, and at the end instead of averaging we use the sum of all learners to make a prediction. the popular choice for weak learners is trees (relatively small ones) so we will talk about them.<br>
in general, there are two main approaches to boosting trees that differ in how we specify our mistakes and how we use trees as weak learners. so let's talk about them separately.

### Adaptive Boosting or **AdaBoost**

In **AdaBoost**, we are using **Stumps** as weak learners. stumps are trees with just one node and 2 leaves. so in each step, we choose the best feature and threshold and separate our data using them. each sample has its weight (equal weight at first), if we make mistake in classifying our sample we have to somehow increase its weight so that the next stumps pay more attention to them. in contrast, we will decrease the weight of samples that classify correctly. there is one more thing we have to specify, and that is how good our stump doing in each iteration so we need to assign some number to it so at the end, in the prediction phase, we more rely on stumps with higher performance. this number called **amount of say** or **stump performance**. lets see how we can wright Adaboost Algorithm.

#### AdaBoost Algorithm

1. Give each sample a weight that indicates how important it is to be correctly classified. at the start, all the samples get the same weight, and that makes the samples all equally important.<br> 
${Weights} = \frac{1}{Total Number Of Samples}\$ <br>
2. Create a stump with the lowest criteria
3. Now we need to determine how much say this stump will have in the final classification, based on how well it classified the samples.<br>
    * Total Error (TE) for a Stump = sum of the weights associated with the incorrectly classified samples.<br>
    
    * Amount of say of a stump = $\frac{1}{2}\ ln\frac{1-TE}{TE}\$<br>
    
4. modify sample weights: we have to increase the weights of samples that the stump misclassified and decrease others.
    * for increasing, new sample weight = sample weight.${e}^{Amount Of Say}$<br>
    * for decreasing, new sample weight = sample weight.${e}^{-Amount Of Say}$<br>
5. now normalize weights so they add up to 1. we divide all the weights by the sum of all the weights. so these weights are our new sample weights for our next stump.
6. now we perform step 2 and use weighted criteria (like weighted Gini) for determining how to split our new stump. weighted Gini will put more emphasis on the sample that has more weight i.e misclassified in the previous stump.
7. we continue making stumps with these 6 steps until we reach our maximum number of stumps or our tolerance. so in the end, we have some stumps with a specific amount of say.
8. at the prediction phase new sample comes, and we check it with all of our stumps, some of them will classify it as 1 and some will classify it as 0, so we add all the amount of say that classify it as 1 and all of them that classify it as 0, and pick maximum as our prediction.<br>
**NOTES:**
   * because all the sample weights add up to 1, so total error always will be between 0 for the perfect stump and 1 for the horrible stump.
   * when stump does a good job, and total error is small, the amount of say will be a relatively large positive value.<br>
     if the total error is 0.5 (stump doing 50/50), the amount of say will be 0.<br>
     and when stump does a terrible job and total error is close to 1, the amount of say will be a relatively large negative value.
   * if the total error is equal to 1 or 0, the amount of say will be $+\infty\ , -\infty\$, so in practice, a small amount of error term is added to prevent this from happening.
   * Alternatively, instead of using weighted criteria, we can make a new collection of samples that contains duplicate copies of the samples with larger sample weights (same size as our original dataset). so:
        * we create bins with our weights by adding them one by one. so instead of weights now we have a range of weights for each sample (a sample with a bigger weight will have a wider bin).
        * we start with an empty dataset that has the same size as our original dataset.
        * pick a random number between 0 and 1.
        * we see where that number falls in our bins and pick that sample and put it in our new dataset.
        * we repeat last two steps until we fill our new dataset. what will happen here is that samples with larger weights will add multiple times in our new dataset.
        * now get rid of the original dataset, and use our new dataset. here we will give all the samples the same weight again.
        * now pick the stump with the lowest criteria value and do the rest of the steps.

### Gradient Boosting

In **Gradient Boosting** instead of using stumps we will use a bigger tree but still, we will restrict the size of trees. also instead of bolding our misclassified samples using weights, we will use something else called **Residuals** which are the difference between predicted values and real values. so in each tree, we will try to predict these residuals (as our mistake to overcome). there is one more thing we should do and it is scaling our tree in each iteration because we don't want to fully rely on our tree in each step in other words we just want some of its information in each step, so we scale them with **learning rate**. the learning rate is some number between 0,1 and constant so all the trees will have the same learning rate.. so we will have something like this:<br>
$$
{f_1(x)}\approx{y}
$$
$$
{f_2(x)}\approx{y}-\alpha{f_1(x)}
$$
$$
{f_3(x)}\approx{y}-\alpha{f_1(x)}-\alpha{f_2(x)}
$$
$$
{.}
$$
$$
{.}
$$
$$
{.}
$$

so if we do it long enough our right-hand side (residuals) will be close to zero and our functions **together** will approximate y more closely. although gradient boosting algorithms for regression and classification have much in common but they have some minor differences so we will talk about them separately.

#### Gradient Boosting Algorithm for Regression

1. calculate average of output value ${y}$, $\bar{y}$. this is our first attempt to predict everyone's, but that's not good enough and we want to do better.<br>

2. Calculate the error we make by assuming $\bar{y}$ as our answer. so the error is the difference between observed ${y}$ and $\bar{y}$.so we will have:<br>
    ${Residuals} = {y} - \bar{y}$

3. now build a tree to predict residuals. in the result, we will have some leaves that contain residuals and because we restrict our tree from growing, so the leaves will have multiple values of residuals, so we will average each leaf and have one value for each leaf. what we have now is ${m}$ samples that connect to 8-32 leaves that output one value (average of our residuals in each leaf).
4. now define learning rate $\alpha$
5. new predicted ${y}$ for each sample will be:

    ${y}_{Predicted} = \bar{y} + \alpha$ (value of leaf that connected to this sample)<br>

6. Calculate new residuals with new predicted ${y}$ like step 2.
7. Again perform step 3 (in practice branches of the tree can be the same or different, but it is usually different)
8. ${y}_{Predicted} = \bar{y} + \alpha$ (Tree1 + Tree2 + ... )<br>
9. Perform steps 6-8 until we reach the maximum step that we define or adding more trees doesn't significantly reduce the size of residuals.
10. in the prediction phase new sample comes, so:<br>

    ${y}_{Prediction} = \bar{y} + \alpha$ (residual that connects this sample in tree 1 + residual that connects this sample in tree 2 + ... )<br>

#### Gradient Boosting Algorithm for Classification

1. Guess initial prediction using log odds and convert it to probability using a logistic function.

    ${Log Odds} = \ln{\frac{class 1}{class 0}}$
    
    Probability of class 1 = $\frac{\exp^{Log Odds}}{1 + \exp^{LogOdds}}$

2. Calculate Residuals:

    ${Residual}$ = (Observed value Probability - Predicted Probability)<br>
    Observed Probability = 0 and 1<br>
    Predicted Probability = here is the value from step 1 but will change later<br>
3. build a tree to predict residuals just like regression. here again, we limit grow of our tree.
4. now we have to calculate the output of each leaf, even leaf with one residual, with this formula:<br>

    output of leaves = $\frac{\sum{Residual_{i}}}{\sum{Previous Probability_{i}) \times (1 - Previous Probability_{i})}}$<br>
    
   in the first tree, previous probabilities are the same for all of the residuals, but later in other trees, it will be different. now every leaf has a single output.<br>
5. define learning rate $\alpha$
6. for each sample we calculate a new ${LogOdds}$ prediction.<br>
    
    new ${LogOdds}$ Prediction = ${LogOdds}$ Prediction (from step 1) + $\alpha$ (output value for each sample)
    
7. Convert new ${LogOdds}$ Prediction to probability using a logistic function. now all the samples have new probability and they are not the same.
8. Calculate residuals with new predicted probability (like step 2).
9. Build a tree to predict new residuals (like step 3).
10. Calculate the output value of leaves (like step 4).
11. new ${LogOdds}$ Prediction for each sample:<br>
    
    new ${LogOdds}$ Prediction = ${LogOdds}$ Prediction (from step 1) + $\alpha$ (output value for each sample in tree 1 + output value for each sample in tree 2 + . . . )<br>

12. Perform steps 7 - 11 until we reach the maximum number of trees or the residuals get super small.
13. in the prediction phase, a new sample comes:<br>
    
    ${LogOdds}$ Prediction = ${LogOdds}$ Prediction (from step 1) + $\alpha$ (output value for each sample in tree 1 + output value for each sample in tree 2 + . . . )<br>
    
    Probability Prediction = ${logistic({LogOdds})}$ Prediction<br>
    
    if the predicted probability is bigger than some threshold (default 0.5) classify as 1 else classify as 0.
    
    

#### Other Variants of Gradient Boosting

what we discussed before, is the raw gradient boost method. by some modifications we can get faster and more accurate algorithms like **XGBoost**, **LightGBM**, and **CatBoost**. let's talk about some of these modifications:<br>
   * Have a unique way for create trees
   * instead of averaging or counting, they optimize all the values in leaves. they also use regularization and second-order derivatives for optimization.
   * instead of sorting and searching threshold for each split, they use binning (approximate algorithm for split finding). by using this algorithm we go from O(NlogN) to O(N).
   * Aggressive sub-sampling and row sampling for creating each tree to prevent overfitting.
   * parallel learning (using multi-core computation)
   * and some other modifications

#### Tuning Gradient Boost methods

* Number of trees
* Learning Rate
* Typically strong pruning via maximum depth
* Maximum feature
* Column subsampling, row subsampling
* Regularization in leaf (typically using L2 regularization)
* we can use early stopping maybe by fixing the learning rate (i don't like early stopping even in gradient descent base models like neural networks :) ).

#### Implementing XGBoost

we are going to solve our breast cancer problem using the XGBoost algorithm but not with the sklearn package. instead, we are going to use the xgboost package. so be sure to install the package first using any package manager like `pip` or `conda`.<br>
it is more accurate and faster than the implementation in scikit learn and also has some other features like supporting categorical variables, supporting multi-core computation, and also supporting GPU. so let's use it :).
for more information visit https://xgboost.readthedocs.io/en/stable/.

In [45]:
from xgboost import XGBClassifier #scikit learn implementation in xgboost package but in general it has its own format
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [46]:
data,target= load_breast_cancer(return_X_y=True) 
X_train, X_test, y_train, y_test = train_test_split(data,target, test_size=0.2, random_state=42,stratify=target)

In [47]:
xgb = XGBClassifier(n_estimators=100,max_depth=3,learning_rate=0.1)
xgb.fit(X_train,y_train)
xgb.score(X_test,y_test)

0.956140350877193

## Recap of Tree Based Models

* Model non-linear relationships.
* Doesn't care about scaling, and supports categorical data.
* Single tree: very interpretable (if small).
* Random Forests are very robust and have good benchmarks (always a good model, so always try it).
* Gradient Boosting often gets the best performance but with careful tuning.
* in general Boosting methods are harder to tune compared to RF.
* in RF more trees are always better but in GB number of trees is strongly dependent on the learning rate.
* by using a high number of trees in GB, we can overfit our model.
* not good when we deal with wide or sparse datasets.